# Blue Brain BioExplorer

![](./images/bioexplorer_banner.png)

## Script to create a visualization of the Spike

In [ ]:
from bioexplorer import BioExplorer, Protein, Sugars, Quaternion
be = BioExplorer('localhost:5000')
be.reset()

In [ ]:
# Resources
resource_folder = '../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
complex_folder = resource_folder + 'pdb/glycans/complex/'

protein_representation = BioExplorer.REPRESENTATION_ATOMS_AND_STICKS

In [ ]:
# Protein
spike = Protein(
    sources=[pdb_folder + '6vyb.pdb'],
    load_hydrogen=False,
    load_non_polymer_chemicals=False)

name = 'Spike'
status = be.add_protein(
    name=name, protein=spike,
    representation=protein_representation)

## Protein information (AA sequences and glycosylation sites)

In [ ]:
be.get_protein_amino_acid_information(assembly_name=name, name=name)

In [ ]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_ATOMS,
    palette_name='Set1', palette_size=2)

## Visualization of amino acids sequence

In [ ]:
status = be.set_protein_amino_acid_sequence_as_range(
    assembly_name=name, name=name, amino_acid_range=[500,750])

status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
    palette_name='Set1', palette_size=2)

In [ ]:
aa_sequence = 'ANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRD'

status = be.set_protein_amino_acid_sequence_as_string(
    assembly_name=name, name=name, amino_acid_sequence=aa_sequence)

status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
    palette_name='Set1', palette_size=2)

## Visualization of glycosylation sites

In [ ]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_GLYCOSYLATION_SITE,
    palette_name='Set1', palette_size=2)

## Visualization of functional regions

In [ ]:
# RGB color palette for amino acid indices
grey = [0.2, 0.2, 0.2]
dark_green = [0.0, 0.5, 0.0]
light_green = [0.0, 1.0, 0.0]
red = [1.0, 0.0, 0.0]
green = [0.0, 1.0, 0.0]
blue = [0.0, 0.0, 1.0]
yellow = [1.0, 1.0, 0.0]
cyan = [1.0, 0.0, 1.0]

region_indices_and_colors = [
    [   1, grey ], [   16, blue], [  306, grey], [  330, green], [438, dark_green], 
    [ 507, green], [  522, grey], [  816, red ], [  835, grey ], [908, yellow],
    [ 986, grey ], [ 1076, cyan], [ 1274, grey], [ 2000, grey ]
]

# Build a palette according to region colors
palette = list()
for index in range(len(region_indices_and_colors)-1):
    for i in range(region_indices_and_colors[index + 1][0] - 
                   region_indices_and_colors[index][0]):
        palette.append(region_indices_and_colors[index][1])

# Apply palette to other chains
status = be.set_protein_color_scheme(
    assembly_name=name, name=name, 
    color_scheme=be.COLOR_SCHEME_ATOMS,
    palette_name='Set1', palette_size=5)

# Apply palette to region color scheme, only for chain #2
status = be.set_protein_color_scheme(
    assembly_name=name, name=name, 
    color_scheme=be.COLOR_SCHEME_REGION, palette=palette,
    chain_ids=[2])

## Add glycans to protein

In [ ]:
glycan_folder = pdb_folder + 'glycans/'
complex_paths = [glycan_folder + 'complex/5.pdb', glycan_folder + 'complex/15.pdb',
                 glycan_folder + 'complex/25.pdb',glycan_folder + 'complex/35.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/20.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/1.pdb']

### High mannose

In [ ]:
high_mannose_glycans = Sugars(
    assembly_name=name, name=be.NAME_GLYCAN_HIGH_MANNOSE, 
    protein_name=name, source=high_mannose_paths[0], 
    site_indices=[80,141,253,622,728,736,820,1093],
    representation=protein_representation)
status = be.add_glycans(high_mannose_glycans)

### Hybrid

In [ ]:
hybrid_glycans = Sugars(
    assembly_name=name, name=be.NAME_GLYCAN_HYBRID, 
    protein_name=name, source=hybrid_paths[0], 
    site_indices=[657+19],
    representation=protein_representation)
status = be.add_glycans(hybrid_glycans)

### Complex

In [ ]:
complex_glycans = Sugars(
    assembly_name=name, name=be.NAME_GLYCAN_COMPLEX, 
    protein_name=name, source=complex_paths[0],
    site_indices=[36,93,168,184,301,350,362,635,1117,1153,1177,1192,1213],
    representation=protein_representation)
status = be.add_glycans(complex_glycans)

### O-Glycans

In [ ]:
for index in [323, 325]:
    o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + \
        str(index)
    o_glycan = Sugars(
        assembly_name=name, name=o_glycan_name,
        source=o_glycan_paths[0], protein_name=name, 
        representation=protein_representation, site_indices=[index])
    be.add_sugars(o_glycan)

## Materials

In [ ]:
be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_NONE)

## Rendering settings

In [ ]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 3
params.gi_weight = 0.4
params.gi_distance = 500
params.shadows = 0.8
params.soft_shadows = 0.3
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 3
be.core_api().set_renderer_params(params)

## Snapshot

In [ ]:
from braynsmediamaker import MovieMaker
mm = MovieMaker(be.core_api())

In [ ]:
mm.create_snapshot(
    path='/home/favreau/6vyb.png',
    size=[512,512], samples_per_pixel=64,
    exportIntermediateFrames=False
)